# Framework tests

## Sonification class interactive tests

In [1]:
import pandas as pd
import panson as ps
import os
import time

FILE_DIR = '../media/files'
OUT_DIR = os.path.join(FILE_DIR, 'processed')

In [2]:
df = pd.read_csv(os.path.join(OUT_DIR, "phone.csv"), sep=r',\s*', engine='python')
df.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,0.191636,0.161091,-0.968156,-0.110604,0.145726,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,0,0.033,0.98,1,0.187249,0.194212,-0.962922,-0.122492,0.179005,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3,0,0.067,0.98,1,0.178678,0.186284,-0.966112,-0.117601,0.177079,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4,0,0.100,0.98,1,0.191069,0.163455,-0.967871,-0.118753,0.161861,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,5,0,0.133,0.98,1,0.184812,0.175222,-0.967027,-0.119328,0.167372,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [3]:
import sc3nb as scn

In [6]:
# start scsynth
sc = scn.startup(start_sclang=False)
# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

<IPython.core.display.Javascript object>

Booting SuperCollider Server... [scsynth | start reading ]
[scsynth]  Found 0 LADSPA plugins
[scsynth]  JackDriver: client name is 'SuperCollider'
[scsynth]  SC_AudioDriver: sample rate = 48000.000000, driver's block size = 1024
[scsynth]  SuperCollider 3 server ready.
Done.
[scsynth]  JackDriver: max output latency 42.7 ms


In [5]:
sc.exit()

Quitting SCServer... [scsynth | reached EOF ]
Done.


In [14]:
au4_node_id = 1001

class AU04ContinuousSonification(ps.Sonification):
    
    def initialize(self):
        return []
    
    def start(self):
        return [
            scn.OSCMessage("/s_new", ["s2", au4_node_id, 0, 0, "amp", 0])
        ]
    
    def stop(self):
        return [
            scn.OSCMessage("/g_freeAll", [0])
        ]
    
    def process(self, row):
        msgs = []
        
        # only "max" should be enough (to clip the top part to 0.3)
        amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")
        # map the intensity of the AU in one octave range
        freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        
        msgs.append(scn.OSCMessage("/n_set", [au4_node_id, "amp", amp, "freq", freq]))
        
        return msgs

In [15]:
son = AU04ContinuousSonification()

### Realtime usage

#### Offline

In [16]:
# send synthdefs
for msg in son.initialize():
    sc.server.send(msg)

**sc.server.bundler vs Bundler**
* sc.server.bundler: adds server.latency to the timestamp (also the absolute one)
* Bundler: does not add anything to the timestamp

In [10]:
# by default is 0
sc.server.latency = 0.2

In [17]:
t0 = time.time()

# instantiate synths asap
for msg in son.start():
    sc.server.send(msg)

# wait one control cycle to be sure that synths are instantiated
# time.sleep(1 / 44100 * 64)


# iterate over dataframe rows
for _, row in df.iterrows():
    
    msgs = son.process(row)
    
    # bundle and send
    with sc.server.bundler(t0 + row.timestamp) as bundler:
        # the bundle will play at t0 + timestamp
        
        # bundle
        for msg in msgs:
            bundler.add(msg)
    
    # sleep for the missing time
    waiting_time = t0 + row.timestamp - time.time()
    
    if waiting_time > 0:
        time.sleep(waiting_time)

        
# instantiate synths asap
for msg in son.stop():
    sc.server.send(msg)

Error at SCServer('127.0.0.1', 57110) pid=16700 from scsynth: ('/fail', '/n_set', 'Node 1001 not found')
Error at SCServer('127.0.0.1', 57110) pid=16700 from scsynth: ('/fail', '/n_set', 'Node 1001 not found')
Error at SCServer('127.0.0.1', 57110) pid=16700 from scsynth: ('/fail', '/n_set', 'Node 1001 not found')
Error at SCServer('127.0.0.1', 57110) pid=16700 from scsynth: ('/fail', '/n_set', 'Node 1001 not found')
Error at SCServer('127.0.0.1', 57110) pid=16700 from scsynth: ('/fail', '/n_set', 'Node 1001 not found')


[scsynth]  FAILURE IN SERVER /n_set Node 1001 not found
[scsynth]  FAILURE IN SERVER /n_set Node 1001 not found
[scsynth]  FAILURE IN SERVER /n_set Node 1001 not found
[scsynth]  FAILURE IN SERVER /n_set Node 1001 not found
[scsynth]  FAILURE IN SERVER /n_set Node 1001 not found
[scsynth | reached EOF ]


In [12]:
sc.server.free_all()

In [13]:
with sc.server.bundler(t0 + row.timestamp) as bundler:
    sc.server.free_all()
    
bundler.messages()

{1640803865.0667663: [<OSCMessage("/g_freeAll", [0])>,
  <OSCMessage("/g_new", [1, 0, 0])>,
  <OSCMessage("/g_new", [67108865, 0, 0])>,
  <OSCMessage("/g_new", [134217729, 0, 0])>,
  <OSCMessage("/g_new", [201326593, 0, 0])>,
  <OSCMessage("/g_new", [268435457, 0, 0])>,
  <OSCMessage("/g_new", [335544321, 0, 0])>,
  <OSCMessage("/g_new", [402653185, 0, 0])>,
  <OSCMessage("/g_new", [469762049, 0, 0])>]}

[scsynth]  late 17.254654408
[scsynth]  late 17.254654408
[scsynth]  late 17.254654408
[scsynth]  late 17.254654408
[scsynth]  late 17.254654408
[scsynth]  late 17.254654408
[scsynth]  late 17.254654408
[scsynth]  late 17.254654408
[scsynth]  late 17.254654408


## Thread behaviour

The threads outputs results in the lastly executed cell.

The cell return when the main thread finishes.

In [11]:
import threading
import time

def worker():
    print(threading.current_thread().getName(), 'Starting')
    time.sleep(10)
    print(threading.current_thread().getName(), 'Exiting')


def my_service():
    print(threading.current_thread().getName(), 'Starting')
    time.sleep(5)
    print(threading.current_thread().getName(), 'Exiting')


t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker)  # use default name

w.start()
w2.start()
t.start()

# # wait until threads terminate
# w.join()
# w2.join()
# t.join()

worker Starting
Thread-16 Starting
my_service Starting


In [12]:
print('test')

test
my_service Exiting
Thread-16 Exiting
worker Exiting


## Misc

Save frame information as csv as expected by the videoviewer

In [ ]:
timestamps = df.timestamp.to_numpy()
np.savetxt('phone.avi.csv', timestamps, delimiter=',')

### Video

#### ipywidgets

In [15]:
from ipywidgets import Video
video = Video.from_file('NRT_videos/phone-processed-son.mp4')
video

True

In [11]:
from ipywidgets import Image
image = Image()

#### PyQt

The following cell enable PyQt5 event loop integration. This is done by opening a QApplication for your notebook. This means that we have to avoid creating QApplication objects in our code (and consequenctly to start the event loop).
* https://stackoverflow.com/questions/30606462/closing-a-pyqt-widget-in-ipython-notebook-without-using-sys-exit

We can run the following magic as many times as we want, but we have to avoid creating QApplications in our code.

In [2]:
%gui qt5

from PyQt5 import QtWidgets, QtCore
import pyqtgraph as pg

In [11]:
QtCore.QCoreApplication.instance()

The following disables IPython GUI event loop integration.

In [24]:
# %gui

In [6]:
# # GUI initialization
# window = QtWidgets.QWidget()

# imggv = pg.GraphicsView()
# viewbox = pg.ViewBox()
# viewbox.setAspectLocked()
# viewbox.invertY(True)

# imggv.setCentralItem(viewbox)

# img = pg.ImageItem(np.zeros((100, 100, 3)))  # Todo: 3 -> channel variable
# viewbox.addItem(img)

# layout = QtWidgets.QGridLayout()
# layout.addWidget(imggv, 0, 0, 4, 4)
# window.setLayout(layout)

# # window always on top
# window.setWindowFlag(QtCore.Qt.WindowStaysOnTopHint)

# window.show()

In [12]:
from videoviewer import NBVideoViewer

vv = NBVideoViewer()
vv.start()

In [13]:
import skvideo
import skvideo.io

frames = skvideo.io.vread('files/processed/phone.avi')
frames.shape

(332, 1280, 720, 3)

In [8]:
# frame = frames[0, ::5, ::5, [0,1,2]].T
# img.setImage(frame)

In [14]:
import ipywidgets as widgets

In [15]:
au4_node_id = 1234
# instantiate synths
sc.server.msg("/s_new", ["s2", au4_node_id, 0, 0, "amp", 0])

In [ ]:
sc.server.free_all()

In [16]:
max_ = 332

# out = widgets.Output()

def on_change(change):
    # with out:
    #    print(change.new)
    
    idx = change.new
    
    row = df.iloc[idx]

    # only "max" should be enough (to clip the top part to 0.3)
    amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")   # TODO: exponential mapping
    # map the intensity of the AU in one octave range
    freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))

    with sc.server.bundler(0.1) as bundler:
        sc.server.msg("/n_set", [au4_node_id, "amp", amp, "freq", freq], bundle=True)

    frame = frames[idx, ::5, ::5, [0,1,2]]
    vv.update(frame)
    
def on_play_stop(change):
    
    # when the video is paused
    if change.now == False:
        # silence all the synths
        with sc.server.bundler() as bundler:
            sc.server.msg("/n_set", [au4_node_id, "amp", 0], bundle=True)
    
        
slider = widgets.IntSlider(max=max_)

# can't play data with unregolar sampling rate
play = widgets.Play(
    max=max_,
    step=1,
    interval=33,
    disabled=False
)

play.observe(on_change, names='value')
play.observe(on_play_stop, names='_playing')

widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])

In [ ]:
def sonify_row(df_row):
    pass

In [20]:
def sonification_callback(event):
    frame_idx = event.cls.value
    
#     sonify_row(df.iloc[frame_idx])
    img.setImage(frames[frame_idx, ::5, ::5, [0,1,2]].T)